<a href="https://colab.research.google.com/github/SandeeeeeeeeepDey/Kaggle_noob_to_pro/blob/main/Titanic_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Kaggle_Competition Jump start



##The Challenge
The sinking of the Titanic is one of the most infamous shipwrecks in history.

On April 15, 1912, during her maiden voyage, the widely considered “unsinkable” RMS Titanic sank after colliding with an iceberg. Unfortunately, there weren’t enough lifeboats for everyone onboard, resulting in the death of 1502 out of 2224 passengers and crew.

While there was some element of luck involved in surviving, it seems some groups of people were more likely to survive than others.

In this challenge, we ask you to build a predictive model that answers the question: “what sorts of people were more likely to survive?” using passenger data (ie name, age, gender, socio-economic class, etc).

##Import and Load

In [ ]:
!pip install tensorflow_decision_forests

In [6]:
import numpy as np
import pandas as pd
import os

import tensorflow as tf
import tensorflow_decision_forests as tfdf

In [7]:
train_df= pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Titanic_Kaggle/train.csv")
test_df= pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Titanic_Kaggle/train.csv")
train_df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


##Dataset Preprocessing/Preparation

Make one preprocess function to use on both training set and test set.

In [ ]:
def preprocess(df):
  df = df.copy()

  def normalize_name(x):
    return " ".join([v.strip(""",()[]."'""") for v in x.split(" ")])

  def ticket_number(x):
    return x.split(" ")[-1]

  def ticket_item(x):
    items = x.split(" ")
    if len(items) < 2:
      return "NONE"
    return "_".join(items[0:-1])

  df["Name"] = df["Name"].apply(normalize_name)
  df["Ticket_number"] = df["Ticket"].apply(ticket_number)
  df["Ticket_item"] = df["Ticket"].apply(ticket_item)
  return df

preprocessed_train_df = preprocess(train_df)
preprocessed_test_df = preprocess(test_df)

In [25]:
preprocessed_train_df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Ticket_number,Ticket_item
0,1,0,3,Braund Mr Owen Harris,male,22.0,1,0,A/5 21171,7.2500,NaN,S,21171,A/5
1,2,1,1,Cumings Mrs John Bradley Florence Briggs Thayer,female,38.0,1,0,PC 17599,71.2833,C85,C,17599,PC
2,3,1,3,Heikkinen Miss Laina,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,3101282,STON/O2.
3,4,1,1,Futrelle Mrs Jacques Heath Lily May Peel,female,35.0,1,0,113803,53.1000,C123,S,113803,NONE
4,5,0,3,Allen Mr William Henry,male,35.0,0,0,373450,8.0500,NaN,S,373450,NONE
5,6,0,3,Moran Mr James,male,NaN,0,0,330877,8.4583,NaN,Q,330877,NONE
6,7,0,1,McCarthy Mr Timothy J,male,54.0,0,0,17463,51.8625,E46,S,17463,NONE
7,8,0,3,Palsson Master Gosta Leonard,male,2.0,3,1,349909,21.0750,NaN,S,349909,NONE
8,9,1,3,Johnson Mrs Oscar W Elisabeth Vilhelmina Berg,female,27.0,0,2,347742,11.1333,NaN,S,347742,NONE
9,10,1,2,Nasser Mrs Nicholas Adele Achem,female,14.0,1,0,237736,30.0708,NaN,C,237736,NONE


Remove columns that will surely not be of any use as a feature.
Saves processing time and reduces dimension

In [15]:
input_features = list(preprocessed_train_df.columns)
input_features.remove("Survived")
input_features.remove("Ticket")
input_features.remove("PassengerId")

print(input_features)

['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked', 'Ticket_number', 'Ticket_item']


## Pandas dataframe to Tensor Dataset


work with python, but at the end use Tensors

In [19]:
def tokenize_names(features, labels=None):
  """TFDF can only use tokens natively, so convert names to tokens"""
  features["Name"] = tf.strings.split(features["Name"])
  return features, labels

train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_train_df,label="Survived").map(tokenize_names)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_train_df,).map(tokenize_names)

##Train Models

###Train with Default Params

In [22]:
model = tfdf.keras.GradientBoostedTreesModel(
    random_seed=1234
)
model.fit(train_ds)

Use /tmp/tmpj8adlrxi as temporary training directory
Reading training dataset...
Training dataset read in 0:00:03.541925. Found 891 examples.
Training model...
Model trained in 0:00:00.231355
Compiling model...
Model compiled.


In [23]:
self_evaluation = model.make_inspector().evaluation()

In [24]:
print(f"Acc:{self_evaluation.accuracy} Loss:{self_evaluation.loss}")

Acc:0.804347813129425 Loss:0.8973425030708313


In [30]:
model.summary()

Model: "gradient_boosted_trees_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "GRADIENT_BOOSTED_TREES"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (11):
	Age
	Cabin
	Embarked
	Fare
	Name
	Parch
	Pclass
	Sex
	SibSp
	Ticket_item
	Ticket_number

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1.           "Sex"  0.805245 ################
    2.           "Age"  0.372369 #####
    3.          "Fare"  0.273026 ##
    4.          "Name"  0.187307 
    5.        "Pclass"  0.180805 
    6.   "Ticket_item"  0.178416 
    7. "Ticket_number"  0.178366 
    8.         "Parch"  0.177684 
    9.      "Embarked"  0.176070 
   10.         "SibSp"  0.172557 

Variable Importance: NUM_AS_ROOT:
    1.  "Sex" 34.000000 ##########

###Improved Default Params

In [27]:
model = tfdf.keras.GradientBoostedTreesModel(
    verbose = 0,
    features = [tfdf.keras.FeatureUsage(name=n) for n in input_features],
    exclude_non_specified_features=True,
    min_examples = 1,
    categorical_algorithm="RANDOM",
    shrinkage=0.05,
    split_axis="SPARSE_OBLIQUE",
    sparse_oblique_normalization="MIN_MAX",
    sparse_oblique_num_projections_exponent = 2.0,
    num_trees=2000,
    random_seed=1234
)

In [28]:
model.fit(train_ds)

In [29]:
self_evaluation = model.make_inspector().evaluation()
print(f"Acc:{self_evaluation.accuracy} Loss:{self_evaluation.loss}")

Acc:0.75 Loss:1.0462466478347778


In [31]:
model.summary()

Model: "gradient_boosted_trees_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "GRADIENT_BOOSTED_TREES"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (11):
	Age
	Cabin
	Embarked
	Fare
	Name
	Parch
	Pclass
	Sex
	SibSp
	Ticket_item
	Ticket_number

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1.           "Sex"  0.805245 ################
    2.           "Age"  0.372369 #####
    3.          "Fare"  0.273026 ##
    4.          "Name"  0.187307 
    5.        "Pclass"  0.180805 
    6.   "Ticket_item"  0.178416 
    7. "Ticket_number"  0.178366 
    8.         "Parch"  0.177684 
    9.      "Embarked"  0.176070 
   10.         "SibSp"  0.172557 

Variable Importance: NUM_AS_ROOT:
    1.  "Sex" 34.000000 ##########